In [1]:
#general imports
%config IPCompleter.greedy=True 
import numpy as np
import time
import zhinst.core
import laboneq
from laboneq.simple import *
import ctypes
import helpers
from picosdk.ps5000a import ps5000a as ps 
from picosdk.functions import adc2mV, assert_pico_ok#, mVadc (can't import for some reason) 
import picoscope_module as pm 
import configparser

DO_EMULATION = False # run in emulation mode by default 

config = configparser.ConfigParser()
config.read('config230823.ini')
with open(config['DataPaths']['descriptor_path'], 'r') as file:
    descriptor = file.read()

# create and connect to session
device_setup = DeviceSetup.from_descriptor(descriptor,server_host="127.0.0.1",server_port="8004", setup_name="ZI_HDAWG")
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

#picoscope 
params = {key: float(value) for key, value in config['PicoscopeParameters'].items()}
pico_sampling_rate = (params['timebase'] - 3) / 62500000
preTriggerSamples=int(np.ceil(params['pretriggersamples']))
postTriggerSamples = int(np.ceil(params['posttriggersamples_val'] / pico_sampling_rate))
maxSamples = preTriggerSamples + postTriggerSamples

NUM_REP = 50

#defining parameters 
times_map = {
    'unload_time': 10e-3, 
    'load_time': 10e-3, 
    'read_time': 10e-3, 
    'trigger_time': 1e-3
}

pulse_map = {
    'unload': [-0.1,-0.1], 
    'load': [-0.1,0.1], 

}

compress_level_pulse_unload_g1=pulse_library.const(uid="compress_level",length=times_map['unload_time'],amplitude=1,can_compress=True)
compress_level_pulse_unload_g2=pulse_library.const(uid="compress_level",length=times_map['unload_time'],amplitude=1,can_compress=True)
compress_level_pulse_load_g1=pulse_library.const(uid="compress_level",length=times_map['load_time'],amplitude=1,can_compress=True)
compress_level_pulse_load_g2=pulse_library.const(uid="compress_level",length=times_map['load_time'],amplitude=1,can_compress=True)

#reverse pulses
compress_level_pulse_unload_g1_r=pulse_library.const(uid="compress_level",length=times_map['unload_time'],amplitude=1,can_compress=True)
compress_level_pulse_unload_g2_r=pulse_library.const(uid="compress_level",length=times_map['unload_time'],amplitude=1,can_compress=True)
compress_level_pulse_load_g1_r=pulse_library.const(uid="compress_level",length=times_map['load_time'],amplitude=1,can_compress=True)
compress_level_pulse_load_g2_r=pulse_library.const(uid="compress_level",length=times_map['load_time'],amplitude=1,can_compress=True)

chandle, status = pm.initialize_picoscope()
status, chARange, chBRange = pm.setup_channels(chandle,status)
status,maxADC = pm.setup_trigger(chandle, chARange, chBRange, params['triggerlevel'],status)

# assign amplitude to received variable
x_p = np.linspace(0.15, -0.1, 26)
y_p = np.linspace(0.15, -0.1, 26)

x_mesh, y_mesh = np.meshgrid(x_p, y_p)
tensor_product = np.column_stack((x_mesh.ravel(), y_mesh.ravel()))

#define pulse shapes
#@pulse_library.register_pulse_functional
#def ramp(x,start=0,stop=1,**_):
#    pulse=start+ (stop-start)*(x+1)/2
#    return pulse


for received_variable in range(len(x_p)*len(y_p)):
    read_amplitude=tensor_product[received_variable]
    compress_level_pulse_read_g1=pulse_library.const(uid="compress_level",length=times_map['read_time'],amplitude=1,can_compress=True)
    compress_level_pulse_read_g2=pulse_library.const(uid="compress_level",length=times_map['read_time'],amplitude=1,can_compress=True)
    compress_level_pulse_read_g1_r=pulse_library.const(uid="compress_level",length=times_map['read_time'],amplitude=1,can_compress=True)
    compress_level_pulse_read_g2_r=pulse_library.const(uid="compress_level",length=times_map['read_time'],amplitude=1,can_compress=True)
    print(read_amplitude)
    # Experiment
    exp = Experiment(
        "Pulse Experiment",
        signals=[
            ExperimentSignal("gate1"),
            ExperimentSignal("gate2"),
        ],
    )
    print(times_map['unload_time'])
    with exp.acquire_loop_rt(
        uid=("pulse"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
    ):
        with exp.section(
            uid=("unload"),
            length=times_map['unload_time'],
            alignment=SectionAlignment.LEFT,
        ):
            exp.play(signal="gate1",pulse=compress_level_pulse_unload_g1, amplitude=pulse_map['unload'][0])
            exp.play(signal="gate2",pulse=compress_level_pulse_unload_g2,amplitude=pulse_map['unload'][1])

        with exp.section(
            uid=("load"),
            length=times_map['load_time'],
            alignment=SectionAlignment.LEFT,
            trigger={"gate1":{"state":1}}, 
        ):
            exp.play(signal="gate1",pulse=compress_level_pulse_load_g1, amplitude=pulse_map['load'][0])
            exp.play(signal="gate2",pulse=compress_level_pulse_load_g2, amplitude=pulse_map['load'][1])

        with exp.section(
            uid=("measure"),
            length=times_map['read_time'],
            alignment=SectionAlignment.LEFT, 
        ): 
            exp.play(signal="gate1",pulse=compress_level_pulse_read_g1, amplitude=read_amplitude[0])
            exp.play(signal="gate2",pulse=compress_level_pulse_read_g2, amplitude=read_amplitude[1])

        with exp.section(
            uid=("rev_measure"),
            length=times_map['read_time'],
            alignment=SectionAlignment.LEFT, 
        ): 
            exp.play(signal="gate1",pulse=compress_level_pulse_read_g1_r, amplitude=-read_amplitude[0])
            exp.play(signal="gate2",pulse=compress_level_pulse_read_g2_r, amplitude=-read_amplitude[1])
        
        with exp.section(
            uid=("rev_load"),
            length=times_map['load_time'],
            alignment=SectionAlignment.LEFT,
             
        ):
            exp.play(signal="gate1",pulse=compress_level_pulse_load_g1_r,amplitude=-pulse_map['load'][0])
            exp.play(signal="gate2",pulse=compress_level_pulse_load_g2_r,amplitude=-pulse_map['load'][1])

        with exp.section(
            uid=("rev_unload"),
            length=times_map['unload_time'],
            alignment=SectionAlignment.LEFT,
        ):
            exp.play(signal="gate1",pulse=compress_level_pulse_unload_g1_r, amplitude=-pulse_map['unload'][0])
            exp.play(signal="gate2",pulse=compress_level_pulse_unload_g2_r, amplitude=-pulse_map['unload'][1])


    #shortcut to the logical signal group q0
    lsg = device_setup.logical_signal_groups["q0"].logical_signals

    #define signal map
    map_signals ={
        "gate1" : lsg["fg4_line"],
        "gate2" : lsg["fg6_line"]
    }

    noOfCaptures = NUM_REP
    status,timeIntervalns = pm.capture_rapid_data_block(chandle, preTriggerSamples, postTriggerSamples, int(params['timebase']), maxSamples,noOfCaptures)


    # set experiment calibration and signal map
    exp.set_signal_map(map_signals)
        
    print("Loaded exp")
    session.run(exp)
    print("Running exp finished")

    # # Check for data collection to finish using ps5000aIsReady
    status = pm.check_ready(chandle, status)

    source = ps.PS5000A_CHANNEL["PS5000A_CHANNEL_B"]
    buffersMax, buffersMin = pm.create_rapid_buffer(chandle, source, maxSamples, noOfCaptures)
    pm.getValuesRapid(chandle,maxSamples,noOfCaptures)

    # #postprocessing
    # # convert ADC counts data to mV
    cmaxSamples = ctypes.c_int32(maxSamples)

    # # Create time data
    time_stamp=int(time.time())
    time_array = pm.create_time_data(maxSamples, timeIntervalns)
    data_save_path = config['DataPaths']['data_save_path']
    with open(f'{data_save_path}pulses_{time_stamp}_a={read_amplitude}.npy', 'wb') as f:
        np.save(f,time_array)
        for i in range(noOfCaptures):
            np.save(f,adc2mV(buffersMax[i], chARange, maxADC))

    with open(f'{data_save_path}config_{time_stamp}_a={read_amplitude}.txt', 'w') as f:
        f.write(str(times_map)+'\n')
        f.write(str(pulse_map)+'\n')
        #Also include here later the whole pulse!!
            
    # Stop the scope
    status = pm.stop_picoscope(chandle,status)

    # Close unit Disconnect the scope 
pm.close_picoscope(chandle,status)

c:\Users\B15measure2\anaconda3\envs\laboneq\lib\site-packages\laboneq\dsl\device\_device_setup_generator.py:1104: FutureWarning: 'instrument_list' section is deprecated in setup descriptor, use 'instruments' instead.
  warnings.warn(


2023.08.24 10:51:51.766 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\B15measure2\ZI_HDAWG_measurements\laboneq_output\log
2023.08.24 10:51:51.768 laboneq.controller.controller  INFO   VERSION: laboneq 2.11.0
2023.08.24 10:51:51.771 laboneq.controller.devices.device_collection INFO   Connecting to data server at 127.0.0.1:8004
2023.08.24 10:51:52.084 laboneq.controller.communication INFO   Connected to Zurich Instruments LabOne Data Server version 23.02 at 127.0.0.1:8004
2023.08.24 10:51:53.023 laboneq.controller.devices.device_collection INFO   Configuring the device setup
2023.08.24 10:51:53.207 laboneq.controller.devices.device_collection INFO   The device setup is configured
[0.15 0.15]
0.01
Loaded exp
2023.08.24 10:51:54.538 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.08.24 10:51:54.544 laboneq.compiler.scheduler.scheduler INFO   Schedule compl